<a href="https://colab.research.google.com/github/WangKehanK/CS505/blob/main/Twitter%20User%20Age%20and%20Gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction on age, gender with M3 inference

m3inference

https://github.com/euagendas/m3inference#existing-json-twitter-data

In [1]:
!pip install m3inference

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 41.4MB 83kB/s 
  Created wheel for rauth: filename=rauth-0.7.3-cp37-none-any.whl size=16055 sha256=2aef011934958a8efe4c1e042dd37f902335b8b4e53b06652a94513368a6970f
  Stored in directory: /root/.cache/pip/wheels/7b/94/5d/81afc278dd5da884a0002563dc4b0fe85f9067a5a40f76f858
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834249 sha256=336e4c921c35d10bd544b40dc7c77b6c730c36b1bd0101ac73e0dd3b9a6d3cbe
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built rauth pycld2


In [2]:
import numpy as np
import pandas as pd
from m3inference import M3Twitter
import json
import os
import pprint

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Twitter_user_handles_to_predict.csv", encoding = "utf-8")
df_labeled = pd.read_csv("/content/drive/My Drive/Twitter_users_labeled_with_age_and_gender.csv", encoding = "latin-1")


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25129 entries, 0 to 25128
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        25129 non-null  int64 
 1   Username  25128 non-null  object
dtypes: int64(1), object(1)
memory usage: 392.8+ KB


In [7]:
df_labeled.head()

,Unnamed: 0,screen_name,user_id,lang,name,location,description,protected,followers_count,friends_count,statuses_count,favourites_count,account_created_at,verified,profile_banner_url,profile_background_url,profile_image_url,user.name,num.tweets.used.Lexicon.prediction,Lexicon.age.prediction,Lexicon.gender.prediction..index.,lexicon.gender.prediction,human.labeled.gender,human.labeled.age
0,1,_____zac_____,4.614412e+08,en,zac ¢,"Maryland, USA",_____Û___È_Ü´Ù,False,208,178,27912,4179,1/11/12 20:19,False,https://pbs.twimg.com/profile_banners/46144118...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1226134911...,@_____zac_____,100.0,27.652434,-1.457167,M,NaN,23.0
1,2,___aleia,7.650000e+17,en,_æ___ dad ___æ_,"Ohio, USA",BLACK. LIVES. MATTER.,False,466,388,3313,23454,8/14/16 19:45,False,https://pbs.twimg.com/profile_banners/76491083...,NaN,http://pbs.twimg.com/profile_images/1271280679...,@___aleia,100.0,24.111464,0.985713,F,NaN,19.0
2,3,___Dals,4.361882e+08,it,_ê_ê___ê__ê_ __,NJ,NaN,False,3744,1465,97563,22499,12/13/11 22:07,False,https://pbs.twimg.com/profile_banners/43618822...,http://abs.twimg.com/images/themes/theme14/bg.gif,http://pbs.twimg.com/profile_images/1267852466...,@___Dals,100.0,30.628957,0.734656,F,F,NaN
3,4,___schaeffer___,1.257110e+09,en,Brenden Schaeffer,The Lou,Culver-Stockton College '20 ¢ Ô_Ô_Ô KM 1548...,False,811,660,26670,72205,3/10/13 14:07,False,https://pbs.twimg.com/profile_banners/12571104...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1268044218...,@___schaeffer___,59.0,35.518352,-3.591586,M,NaN,22.0
4,5,__andresiscool,1.030000e+18,en,Andres Navarro,NaN,NaN,False,245,184,8731,18574,8/27/18 5:21,False,https://pbs.twimg.com/profile_banners/10339476...,NaN,http://pbs.twimg.com/profile_images/1272229718...,@__andresiscool,100.0,21.014222,-1.623685,M,M,NaN


In [8]:
df.head()

,ID,Username
0,1,JayHolz410
1,2,kelechief
2,3,VicSpencer
3,4,ItsAlexDodson
4,5,xopinkvodka6


In [9]:
user_list = df['Username'].tolist()
id_list = df_labeled['user_id'].tolist()

In [10]:
username = user_list[0] # try one user first, we can loop over the entire user list to get all the user detail
m3cachedir = "drive/MyDrive/cachedir"

if not os.path.exists(m3cachedir):
  os.mkdir(m3cachedir)

with open(f'/content/{m3cachedir}/{username}.json', 'w') as json_file:
  # initialization
  m3twitter=M3Twitter()

  # remember to upload Twitter API keys, format as https://github.com/euagendas/m3inference/blob/master/scripts/auth_example.txt
  m3twitter.twitter_init_from_file("/content/drive/MyDrive/auth.txt")

  # you can also use m3twitter.infer_id(id = "....")
  user_detail = m3twitter.infer_screen_name(username)

  # print the user detail
  pprint.pprint(user_detail)
  # save the user detail for future use
  json.dump(user_detail, json_file)


04/06/2021 05:24:23 - INFO - m3inference.m3inference -   Version 1.1.4
04/06/2021 05:24:23 - INFO - m3inference.m3inference -   Running on cpu.
04/06/2021 05:24:23 - INFO - m3inference.m3inference -   Will use full M3 model.
04/06/2021 05:24:24 - INFO - m3inference.m3inference -   Dir /root/m3/models/ does not exist. Creating now.
04/06/2021 05:24:24 - INFO - m3inference.m3inference -   Dir /root/m3/models/ created.
04/06/2021 05:24:24 - INFO - m3inference.m3inference -   Model full_model does not exist at /root/m3/models/full_model.mdl. Try to download it now.
04/06/2021 05:24:24 - INFO - m3inference.utils -   /root/m3/models/full_model.mdl not found in cache, downloading from https://nlp.stanford.edu/~zijwang/m3inference/full_model.mdl to /tmp/tmpshj6qltp
141007KB [01:42, 1373.95KB/s]                           
04/06/2021 05:26:07 - INFO - m3inference.utils -   Model full_model was downloaded to a tmp file.
04/06/2021 05:26:07 - INFO - m3inference.utils -   Copying tmp file to /root/

{'input': {'description': '',
           'id': 'dummy',
           'img_path': '/usr/local/lib/python3.7/dist-packages/m3inference/data/tw_default_profile.png',
           'lang': 'un',
           'name': '',
           'screen_name': ''},
 'output': {'age': {'19-29': 0.2393,
                    '30-39': 0.0793,
                    '<=18': 0.1746,
                    '>=40': 0.5067},
            'gender': {'female': 0.2809, 'male': 0.7191},
            'org': {'is-org': 0.0873, 'non-org': 0.9127}}}


In [11]:
# Create new column to save the predicted value
df["age"] = 0
df["gender"] = "male"
df["org"] = ""

In [ ]:
# Loop over each row to assign the predict value, should be used with M3
for index, row in df.iterrows():
    df.at[index, 'age'] = 1

In [51]:
# Get authentication for tweepy from auth.txt
# auth.txt was uploaded before in order to use m3inference
keys = ''
with open(f'/content/drive/MyDrive/auth.txt', 'r') as reader:
  keys=reader.read()
keys = keys.split('\n')
for i in range(len(keys)):
  keys[i] = keys[i].split('"')[1];

In [35]:
# Setup tweepy
import tweepy
api_key = keys[0]
api_secret = keys[1]
access_token = keys[2]
access_secret = keys[3]
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [41]:
# Try first user
user = df['Username']
userID = user[0]
print(userID)
tweets = api.user_timeline(screen_name=userID,count=100,include_rts = False,tweet_mode = 'extended')

JayHolz410


In [50]:
# Extract full text from tweets
tweet_text = [tweet._json['full_text'] for tweet in tweets]
print(len(tweet_text))

94


TODO: Using BERT to extract fixed feature vectors 

https://github.com/google-research/bert#using-bert-to-extract-fixed-feature-vectors-like-elmo

In [59]:
# drop rows does not have age label in the labeled dataset
age_no_na = df_labeled[df_labeled['human.labeled.age'].notna()]
age_no_na

,Unnamed: 0,screen_name,user_id,lang,name,location,description,protected,followers_count,friends_count,statuses_count,favourites_count,account_created_at,verified,profile_banner_url,profile_background_url,profile_image_url,user.name,num.tweets.used.Lexicon.prediction,Lexicon.age.prediction,Lexicon.gender.prediction..index.,lexicon.gender.prediction,human.labeled.gender,human.labeled.age
0,1,_____zac_____,4.614412e+08,en,zac ¢,"Maryland, USA",_____Û___È_Ü´Ù,False,208,178,27912,4179,1/11/12 20:19,False,https://pbs.twimg.com/profile_banners/46144118...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1226134911...,@_____zac_____,100.0,27.652434,-1.457167,M,NaN,23.0
1,2,___aleia,7.650000e+17,en,_æ___ dad ___æ_,"Ohio, USA",BLACK. LIVES. MATTER.,False,466,388,3313,23454,8/14/16 19:45,False,https://pbs.twimg.com/profile_banners/76491083...,NaN,http://pbs.twimg.com/profile_images/1271280679...,@___aleia,100.0,24.111464,0.985713,F,NaN,19.0
3,4,___schaeffer___,1.257110e+09,en,Brenden Schaeffer,The Lou,Culver-Stockton College '20 ¢ Ô_Ô_Ô KM 1548...,False,811,660,26670,72205,3/10/13 14:07,False,https://pbs.twimg.com/profile_banners/12571104...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1268044218...,@___schaeffer___,59.0,35.518352,-3.591586,M,NaN,22.0
8,9,__drewc,1.050000e+18,en,drew,"New York, USA",_öÂ_öé _öÂ_ö_ _ç´Ù È \r\r26 #NewYork,False,27,128,181,113,10/17/18 4:12,False,https://pbs.twimg.com/profile_banners/10524118...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1090809548...,@__drewc,134.0,24.910635,1.969121,F,NaN,26.0
9,10,__EmilyRice__,3.797155e+09,en,em,"Marble Falls, TX",#TXST22,False,158,190,1219,2596,10/5/15 23:30,False,https://pbs.twimg.com/profile_banners/37971551...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/8415201103...,@__EmilyRice__,100.0,25.191925,2.382856,F,NaN,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,3272,ZoeBerrier,9.020000e+17,en,Zoâ _êâ,"Millersville, PA",MU 2021 (she/her)\r\rQueen of putting lipstick...,False,94,298,858,4428,8/28/17 21:03,False,https://pbs.twimg.com/profile_banners/90227535...,NaN,http://pbs.twimg.com/profile_images/1241199033...,@ZoeBerrier,100.0,22.585143,1.243141,F,F,20.0
3272,3273,ZoeCalamaco,3.214954e+09,no,Zoe _,San Angelo tx/ aspermont tx,Angelo state,False,475,318,6494,7475,4/28/15 0:24,False,https://pbs.twimg.com/profile_banners/32149543...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1277399838...,@ZoeCalamaco,100.0,21.348766,1.939069,F,F,21.0
3274,3275,ZoPeachy,9.890000e+17,en,Zobella Thee Alpha __ê_____´Ù__ ...,"New England/Boston, MA",Harlot for hire. FinDom. 27. Nonbinary. they/t...,False,396,635,3553,2075,4/25/18 3:51,False,https://pbs.twimg.com/profile_banners/98898876...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1153648270...,@ZoPeachy,100.0,34.894953,2.058720,F,NaN,27.0
3277,3278,zupercar1,4.220999e+09,en,Chris Avery _ê___êÈ,"Ann Arbor, MI",Comic Book Artist/ Film Major / I like space.....,False,121,1974,31208,70426,11/18/15 20:57,False,https://pbs.twimg.com/profile_banners/42209994...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1145126143...,@zupercar1,100.0,23.884346,-2.540524,M,NaN,21.0
